In [1]:
import subprocess
subprocess.run(["pip", "install", "pronouncing", "-q"])

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
from datasets import load_dataset
from dataclasses import dataclass, field
import math
import numpy as np
import pronouncing
from collections import defaultdict
import random
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

/home/andrey/anaconda3/envs/A_10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
GPU: NVIDIA GeForce RTX 4070 Ti SUPER


In [2]:
@dataclass
class ResonanceConfig:
    name: str = "base"
    vocab_size: int = 50257
    max_seq_len: int = 256
    embed_dim: int = 768
    n_layers: int = 12
    n_heads: int = 12
    ff_dim: int = 3072
    n_frequencies: int = 32
    resonance_blend: float = 0.3
    resonance_attn_weight: float = 0.1
    dropout: float = 0.1
    batch_size: int = 8
    gradient_accumulation: int = 4
    learning_rate: float = 3e-4
    
    # The key difference
    phonetic_init: bool = False

# Config A: Random phase init (baseline)
config_random = ResonanceConfig(
    name="random_phase",
    phonetic_init=False
)

# Config B: Phonetic phase init (your theory)
config_phonetic = ResonanceConfig(
    name="phonetic_phase", 
    phonetic_init=True
)

# Config C: SMALLER model with phonetic init (the real test)
config_small_phonetic = ResonanceConfig(
    name="small_phonetic",
    embed_dim=512,       # Smaller
    n_layers=8,          # Fewer layers
    n_heads=8,
    ff_dim=2048,
    n_frequencies=32,
    phonetic_init=True
)

def count_params(config):
    embed = config.vocab_size * config.embed_dim
    phase = config.vocab_size * config.n_frequencies
    attn = config.n_layers * (4 * config.embed_dim ** 2)
    ff = config.n_layers * (2 * config.embed_dim * config.ff_dim)
    return embed + phase + attn + ff

print("Models to compare:")
print(f"  A) Random 126M:    {count_params(config_random)/1e6:.0f}M params, random phase init")
print(f"  B) Phonetic 126M:  {count_params(config_phonetic)/1e6:.0f}M params, rhyme-based phase init")
print(f"  C) Phonetic 45M:   {count_params(config_small_phonetic)/1e6:.0f}M params, rhyme-based phase init")
print()
print("Hypothesis: C matches or beats A despite having 1/3 the parameters")

Models to compare:
  A) Random 126M:    125M params, random phase init
  B) Phonetic 126M:  125M params, rhyme-based phase init
  C) Phonetic 45M:   53M params, rhyme-based phase init

Hypothesis: C matches or beats A despite having 1/3 the parameters


In [3]:
def build_rhyme_index(tokenizer, vocab_size):
    """
    Build the rhyme-based index structure.
    This is the core of your compression theory.
    """
    
    print("Building rhyme index...")
    
    # Map each token to its rhyme signature
    token_to_rhyme = {}
    rhyme_to_tokens = defaultdict(list)
    
    # Also track phonetic features for richer structure
    token_to_phones = {}
    
    for token_id in range(vocab_size):
        token_str = tokenizer.decode([token_id]).strip().lower()
        token_clean = ''.join(c for c in token_str if c.isalpha())
        
        if len(token_clean) < 2:
            continue
        
        phones = pronouncing.phones_for_word(token_clean)
        
        if phones:
            rhyme_part = pronouncing.rhyming_part(phones[0])
            token_to_rhyme[token_id] = rhyme_part
            rhyme_to_tokens[rhyme_part].append(token_id)
            token_to_phones[token_id] = phones[0]
    
    # Create hierarchical structure
    # Level 1: Rhyme groups (words that rhyme together)
    # Level 2: Phonetic similarity within groups
    
    rhyme_index = {
        "token_to_rhyme": token_to_rhyme,
        "rhyme_to_tokens": dict(rhyme_to_tokens),
        "token_to_phones": token_to_phones,
        "n_rhyme_groups": len(rhyme_to_tokens),
        "tokens_with_rhyme": len(token_to_rhyme),
    }
    
    # Stats
    print(f"  Tokens with rhyme: {rhyme_index['tokens_with_rhyme']}/{vocab_size}")
    print(f"  Unique rhyme groups: {rhyme_index['n_rhyme_groups']}")
    
    # Largest groups
    sorted_groups = sorted(rhyme_to_tokens.items(), key=lambda x: -len(x[1]))
    print(f"\n  Largest rhyme groups:")
    for rhyme, tokens in sorted_groups[:5]:
        words = [tokenizer.decode([t]).strip() for t in tokens[:8]]
        print(f"    '{rhyme}' ({len(tokens)} words): {words}")
    
    return rhyme_index

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

rhyme_index = build_rhyme_index(tokenizer, config_random.vocab_size)

Building rhyme index...
  Tokens with rhyme: 35431/50257
  Unique rhyme groups: 8045

  Largest rhyme groups:
    'EY1 SH AH0 N' (422 words): ['formation', 'information', 'situation', 'population', 'nation', 'application', 'investigation', 'administration']
    'IY1' (405 words): ['he', 've', 'be', 'ac', 'he', 'ce', 'we', 'de']
    'UW1' (289 words): ['ou', 'to', 'you', 'qu', 'su', 'do', 'who', 'ru']
    'EY1' (281 words): ['re', 're', 'ay', 'se', 'se', 'ok', 'they', 'play']
    'OW1' (257 words): ['ro', 'ow', 'pro', 'go', 'so', 'au', 'so', 'no']


In [4]:
class ResonanceEmbedding(nn.Module):
    """
    Embedding layer with optional phonetic phase initialization.
    
    The key insight: if phonetic_init=True, words that rhyme
    start close in phase-space. This gives the model a structural
    prior that should require fewer parameters to exploit.
    """
    
    def __init__(self, config, rhyme_index=None):
        super().__init__()
        self.config = config
        
        # Semantic embedding
        self.semantic = nn.Embedding(config.vocab_size, config.embed_dim)
        
        # Phase embedding
        self.phase = nn.Embedding(config.vocab_size, config.n_frequencies)
        
        # Phase projection
        self.phase_proj = nn.Linear(config.n_frequencies, config.embed_dim, bias=False)
        
        # Learnable blend
        self.blend = nn.Parameter(torch.full((config.embed_dim,), config.resonance_blend))
        
        # Position embedding
        self.position = nn.Embedding(config.max_seq_len, config.embed_dim)
        
        self.dropout = nn.Dropout(config.dropout)
        
        # Initialize
        self._init_weights()
        
        if config.phonetic_init and rhyme_index:
            self._init_phonetic_phases(rhyme_index)
        else:
            self._init_random_phases()
    
    def _init_weights(self):
        nn.init.normal_(self.semantic.weight, std=0.02)
        nn.init.normal_(self.phase_proj.weight, std=0.02)
        nn.init.normal_(self.position.weight, std=0.02)
    
    def _init_random_phases(self):
        """Standard random initialization"""
        print("  Initializing phases: RANDOM")
        nn.init.normal_(self.phase.weight, std=0.3)
    
    def _init_phonetic_phases(self, rhyme_index):
        """
        Initialize phases so rhyming words are close.
        This is the core of the compression hypothesis.
        """
        print("  Initializing phases: PHONETIC (rhyme-based)")
        
        rhyme_to_tokens = rhyme_index["rhyme_to_tokens"]
        
        # Assign a base phase vector to each rhyme group
        rhyme_to_phase = {}
        for rhyme in rhyme_to_tokens.keys():
            rhyme_to_phase[rhyme] = torch.randn(self.config.n_frequencies) * 0.3
        
        with torch.no_grad():
            # Initialize all to random first
            self.phase.weight.data = torch.randn_like(self.phase.weight) * 0.3
            
            # Then override tokens with known rhymes
            phonetic_count = 0
            for rhyme, tokens in rhyme_to_tokens.items():
                base_phase = rhyme_to_phase[rhyme]
                for token_id in tokens:
                    # Small noise so they're close but not identical
                    self.phase.weight.data[token_id] = base_phase + torch.randn(self.config.n_frequencies) * 0.03
                    phonetic_count += 1
        
        print(f"    Phonetically initialized: {phonetic_count} tokens")
        print(f"    Randomly initialized: {self.config.vocab_size - phonetic_count} tokens")
    
    def get_resonance_matrix(self, token_ids):
        phases = self.phase(token_ids)
        phase_diff = phases.unsqueeze(2) - phases.unsqueeze(1)
        resonance = torch.cos(phase_diff).mean(dim=-1)
        return resonance
    
    def forward(self, token_ids):
        batch_size, seq_len = token_ids.shape
        
        sem = self.semantic(token_ids)
        ph = self.phase(token_ids)
        ph_proj = self.phase_proj(ph)
        
        blend = torch.sigmoid(self.blend)
        embeddings = (1 - blend) * sem + blend * ph_proj
        
        positions = torch.arange(seq_len, device=token_ids.device)
        embeddings = embeddings + self.position(positions)
        
        resonance = self.get_resonance_matrix(token_ids)
        embeddings = self.dropout(embeddings)
        
        return embeddings, resonance

In [5]:
class ResonanceAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.n_heads = config.n_heads
        self.head_dim = config.embed_dim // config.n_heads
        self.scale = self.head_dim ** -0.5
        
        self.qkv = nn.Linear(config.embed_dim, 3 * config.embed_dim, bias=False)
        self.out_proj = nn.Linear(config.embed_dim, config.embed_dim, bias=False)
        self.resonance_weight = nn.Parameter(torch.full((config.n_heads,), config.resonance_attn_weight))
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x, resonance, mask=None):
        B, S, _ = x.shape
        
        qkv = self.qkv(x).reshape(B, S, 3, self.n_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        attn = attn + resonance.unsqueeze(1) * self.resonance_weight.view(1, self.n_heads, 1, 1)
        
        if mask is not None:
            attn = attn.masked_fill(mask == 0, float('-inf'))
        
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        
        out = torch.matmul(attn, v).transpose(1, 2).reshape(B, S, -1)
        return self.out_proj(out)


class ResonanceBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.embed_dim)
        self.attn = ResonanceAttention(config)
        self.ln2 = nn.LayerNorm(config.embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(config.embed_dim, config.ff_dim),
            nn.GELU(),
            nn.Linear(config.ff_dim, config.embed_dim),
            nn.Dropout(config.dropout)
        )
    
    def forward(self, x, resonance, mask=None):
        x = x + self.attn(self.ln1(x), resonance, mask)
        x = x + self.ff(self.ln2(x))
        return x

In [6]:
class ResonanceTransformer(nn.Module):
    def __init__(self, config, rhyme_index=None):
        super().__init__()
        self.config = config
        
        print(f"\nBuilding model: {config.name}")
        
        self.embedding = ResonanceEmbedding(config, rhyme_index)
        self.blocks = nn.ModuleList([ResonanceBlock(config) for _ in range(config.n_layers)])
        self.ln_final = nn.LayerNorm(config.embed_dim)
        self.lm_head = nn.Linear(config.embed_dim, config.vocab_size, bias=False)
        self.lm_head.weight = self.embedding.semantic.weight
        
        self.register_buffer("causal_mask", torch.tril(torch.ones(config.max_seq_len, config.max_seq_len)))
        
        self.apply(self._init_weights)
        self.n_params = sum(p.numel() for p in self.parameters())
        print(f"  Parameters: {self.n_params/1e6:.2f}M")
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, input_ids, labels=None):
        B, S = input_ids.shape
        x, resonance = self.embedding(input_ids)
        mask = self.causal_mask[:S, :S].unsqueeze(0)
        
        for block in self.blocks:
            x = block(x, resonance, mask)
        
        logits = self.lm_head(self.ln_final(x))
        
        result = {"logits": logits}
        if labels is not None:
            result["loss"] = F.cross_entropy(
                logits[:, :-1, :].contiguous().view(-1, self.config.vocab_size),
                labels[:, 1:].contiguous().view(-1),
                ignore_index=-100
            )
        return result

In [7]:
print("Loading OpenWebText (5%)...")
dataset = load_dataset("openwebtext", split="train[:5%]", trust_remote_code=True)
print(f"Loaded {len(dataset)} documents")

class TextDataset(Dataset):
    def __init__(self, documents, tokenizer, max_length=256, max_examples=200000):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.examples = []
        
        print(f"Tokenizing (max {max_examples} examples)...")
        
        for i, doc in enumerate(documents):
            tokens = tokenizer.encode(doc["text"], truncation=False)
            
            for j in range(0, len(tokens) - max_length + 1, max_length):
                self.examples.append(tokens[j:j + max_length])
                if len(self.examples) >= max_examples:
                    break
            
            if len(self.examples) >= max_examples:
                break
            
            if (i + 1) % 20000 == 0:
                print(f"  {i+1} docs, {len(self.examples)} examples")
        
        print(f"Final: {len(self.examples)} examples")
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        tokens = torch.tensor(self.examples[idx], dtype=torch.long)
        return {"input_ids": tokens, "labels": tokens}

# Create dataset - same for all models
train_dataset = TextDataset(dataset, tokenizer, max_length=256, max_examples=200000)
val_dataset = TextDataset(
    dataset.select(range(len(dataset)-5000, len(dataset))),
    tokenizer, max_length=256, max_examples=5000
)

# Will create loaders per-model due to different batch sizes
print(f"\nTrain: {len(train_dataset)} examples")
print(f"Val: {len(val_dataset)} examples")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'openwebtext' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Loading OpenWebText (5%)...


Generating train split: 100%|██████████| 8013769/8013769 [01:02<00:00, 128079.13 examples/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (1217 > 1024). Running this sequence through the model will result in indexing errors


Loaded 400688 documents
Tokenizing (max 200000 examples)...
  20000 docs, 77931 examples
  40000 docs, 156798 examples
Final: 200000 examples
Tokenizing (max 5000 examples)...
Final: 5000 examples

Train: 200000 examples
Val: 5000 examples


In [9]:
def train_model(model, config, train_dataset, val_dataset, n_epochs=2):
    """Train a single model and return final perplexity"""
    
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=0.01)
    
    print(f"\nTraining: {config.name}")
    print(f"  Batches per epoch: {len(train_loader)}")
    
    results = {"train_loss": [], "val_loss": [], "val_ppl": []}
    
    for epoch in range(n_epochs):
        # Train
        model.train()
        train_loss = 0
        start_time = time.time()
        
        for i, batch in enumerate(train_loader):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            
            loss = model(input_ids, labels=labels)["loss"]
            loss.backward()
            
            if (i + 1) % config.gradient_accumulation == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                optimizer.zero_grad()
            
            train_loss += loss.item()
            
            if (i + 1) % 500 == 0:
                print(f"    Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        epoch_time = time.time() - start_time
        avg_train = train_loss / len(train_loader)
        
        # Validate
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                labels = batch["labels"].to(device)
                val_loss += model(input_ids, labels=labels)["loss"].item()
        
        avg_val = val_loss / len(val_loader)
        val_ppl = math.exp(avg_val)
        
        results["train_loss"].append(avg_train)
        results["val_loss"].append(avg_val)
        results["val_ppl"].append(val_ppl)
        
        # Check blend
        blend = torch.sigmoid(model.embedding.blend).mean().item()
        
        print(f"\n  Epoch {epoch+1}/{n_epochs} ({epoch_time/60:.1f} min)")
        print(f"    Train Loss: {avg_train:.4f}")
        print(f"    Val Loss: {avg_val:.4f}")
        print(f"    Val Perplexity: {val_ppl:.2f}")
        print(f"    Embedding blend: {blend:.3f}")
    
    return results

In [10]:
print("="*70)
print("A/B TEST: DOES PHONETIC INIT COMPRESS BETTER?")
print("="*70)

all_results = {}

# Model A: Random phases (126M baseline)
print("\n" + "="*70)
print("MODEL A: Random Phase Init (126M) - BASELINE")
print("="*70)
model_a = ResonanceTransformer(config_random, rhyme_index=None).to(device)
results_a = train_model(model_a, config_random, train_dataset, val_dataset, n_epochs=2)
all_results["random_126M"] = results_a
torch.save(model_a.state_dict(), "model_a_random_126M.pt")
del model_a
torch.cuda.empty_cache()

# Model B: Phonetic phases (126M)
print("\n" + "="*70)
print("MODEL B: Phonetic Phase Init (126M)")
print("="*70)
model_b = ResonanceTransformer(config_phonetic, rhyme_index=rhyme_index).to(device)
results_b = train_model(model_b, config_phonetic, train_dataset, val_dataset, n_epochs=2)
all_results["phonetic_126M"] = results_b
torch.save(model_b.state_dict(), "model_b_phonetic_126M.pt")
del model_b
torch.cuda.empty_cache()

# Model C: Phonetic phases (45M) - THE KEY TEST
print("\n" + "="*70)
print("MODEL C: Phonetic Phase Init (45M) - COMPRESSION TEST")
print("="*70)
model_c = ResonanceTransformer(config_small_phonetic, rhyme_index=rhyme_index).to(device)
results_c = train_model(model_c, config_small_phonetic, train_dataset, val_dataset, n_epochs=2)
all_results["phonetic_45M"] = results_c
torch.save(model_c.state_dict(), "model_c_phonetic_45M.pt")

A/B TEST: DOES PHONETIC INIT COMPRESS BETTER?

MODEL A: Random Phase Init (126M) - BASELINE

Building model: random_phase
  Initializing phases: RANDOM
  Parameters: 125.45M

Training: random_phase
  Batches per epoch: 25000
    Batch 500/25000, Loss: 6.9995
    Batch 1000/25000, Loss: 6.2706
    Batch 1500/25000, Loss: 6.3181
    Batch 2000/25000, Loss: 6.0901
    Batch 2500/25000, Loss: 6.3043
    Batch 3000/25000, Loss: 6.0382
    Batch 3500/25000, Loss: 5.9140
    Batch 4000/25000, Loss: 6.1092
    Batch 4500/25000, Loss: 6.0507
    Batch 5000/25000, Loss: 5.7492
    Batch 5500/25000, Loss: 5.3950
    Batch 6000/25000, Loss: 5.6699
    Batch 6500/25000, Loss: 5.5523
    Batch 7000/25000, Loss: 5.0546
    Batch 7500/25000, Loss: 5.4438
    Batch 8000/25000, Loss: 5.3062
    Batch 8500/25000, Loss: 5.2996
    Batch 9000/25000, Loss: 5.1798
    Batch 9500/25000, Loss: 5.2685
    Batch 10000/25000, Loss: 4.8852
    Batch 10500/25000, Loss: 4.7940
    Batch 11000/25000, Loss: 5.0844
   

In [11]:
print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)

print("\nFinal Perplexity Comparison:\n")
print(f"{'Model':<25} {'Params':<12} {'Val PPL':<12} {'vs Baseline'}")
print("-" * 60)

baseline_ppl = all_results["random_126M"]["val_ppl"][-1]

for name, results in all_results.items():
    ppl = results["val_ppl"][-1]
    
    if "126M" in name:
        params = "126M"
    else:
        params = "45M"
    
    if name == "random_126M":
        vs_baseline = "(baseline)"
    else:
        diff = ((ppl - baseline_ppl) / baseline_ppl) * 100
        vs_baseline = f"{diff:+.1f}%"
    
    print(f"{name:<25} {params:<12} {ppl:<12.2f} {vs_baseline}")

print("\n" + "="*70)
print("HYPOTHESIS TEST:")
print("="*70)

phonetic_45_ppl = all_results["phonetic_45M"]["val_ppl"][-1]
random_126_ppl = all_results["random_126M"]["val_ppl"][-1]

if phonetic_45_ppl <= random_126_ppl * 1.1:  # Within 10%
    print(f"""
✓ SUCCESS: 45M phonetic model achieves {phonetic_45_ppl:.2f} perplexity
  vs 126M random model at {random_126_ppl:.2f} perplexity.
  
  This represents ~2.8x parameter efficiency from phonetic structure.
  
  YOUR THEORY IS SUPPORTED: Rhyme-based indexing compresses better.
""")
else:
    print(f"""
✗ 45M phonetic model ({phonetic_45_ppl:.2f}) did not match 126M baseline ({random_126_ppl:.2f}).
  
  Possible reasons:
  - Need more training epochs
  - Need more/better data
  - Phonetic structure helps but doesn't fully compensate for size
  
  However, compare 126M phonetic vs 126M random to see if phonetic helps at all.
""")

phonetic_126_ppl = all_results["phonetic_126M"]["val_ppl"][-1]
if phonetic_126_ppl < random_126_ppl:
    print(f"""
✓ PARTIAL SUCCESS: At same size (126M), phonetic init beats random:
  Phonetic: {phonetic_126_ppl:.2f}
  Random: {random_126_ppl:.2f}
  Improvement: {((random_126_ppl - phonetic_126_ppl) / random_126_ppl) * 100:.1f}%
  
  Phonetic structure provides measurable benefit.
""")


RESULTS SUMMARY

Final Perplexity Comparison:

Model                     Params       Val PPL      vs Baseline
------------------------------------------------------------
random_126M               126M         63.88        (baseline)
phonetic_126M             126M         63.17        -1.1%
phonetic_45M              45M          70.92        +11.0%

HYPOTHESIS TEST:

✗ 45M phonetic model (70.92) did not match 126M baseline (63.88).
  
  Possible reasons:
  - Need more training epochs
  - Need more/better data
  - Phonetic structure helps but doesn't fully compensate for size
  
  However, compare 126M phonetic vs 126M random to see if phonetic helps at all.


✓ PARTIAL SUCCESS: At same size (126M), phonetic init beats random:
  Phonetic: 63.17
  Random: 63.88
  Improvement: 1.1%
  
  Phonetic structure provides measurable benefit.



In [12]:
# Config for standard transformer (no resonance)
@dataclass
class StandardConfig:
    name: str = "standard_baseline"
    vocab_size: int = 50257
    max_seq_len: int = 256
    embed_dim: int = 768
    n_layers: int = 12
    n_heads: int = 12
    ff_dim: int = 3072
    dropout: float = 0.1
    batch_size: int = 8
    gradient_accumulation: int = 4
    learning_rate: float = 3e-4

class StandardTransformer(nn.Module):
    """Vanilla transformer - no resonance, no phase embeddings"""
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        print(f"\nBuilding model: {config.name}")
        
        # Standard embeddings only
        self.token_embed = nn.Embedding(config.vocab_size, config.embed_dim)
        self.pos_embed = nn.Embedding(config.max_seq_len, config.embed_dim)
        self.dropout = nn.Dropout(config.dropout)
        
        # Standard transformer blocks
        self.blocks = nn.ModuleList([
            StandardBlock(config) for _ in range(config.n_layers)
        ])
        
        self.ln_final = nn.LayerNorm(config.embed_dim)
        self.lm_head = nn.Linear(config.embed_dim, config.vocab_size, bias=False)
        self.lm_head.weight = self.token_embed.weight
        
        self.register_buffer("causal_mask", torch.tril(torch.ones(config.max_seq_len, config.max_seq_len)))
        
        self.apply(self._init_weights)
        self.n_params = sum(p.numel() for p in self.parameters())
        print(f"  Parameters: {self.n_params/1e6:.2f}M")
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, input_ids, labels=None):
        B, S = input_ids.shape
        
        positions = torch.arange(S, device=input_ids.device)
        x = self.token_embed(input_ids) + self.pos_embed(positions)
        x = self.dropout(x)
        
        mask = self.causal_mask[:S, :S].unsqueeze(0)
        
        for block in self.blocks:
            x = block(x, mask)
        
        logits = self.lm_head(self.ln_final(x))
        
        result = {"logits": logits}
        if labels is not None:
            result["loss"] = F.cross_entropy(
                logits[:, :-1, :].contiguous().view(-1, self.config.vocab_size),
                labels[:, 1:].contiguous().view(-1),
                ignore_index=-100
            )
        return result


class StandardAttention(nn.Module):
    """Vanilla attention - no resonance bias"""
    
    def __init__(self, config):
        super().__init__()
        self.n_heads = config.n_heads
        self.head_dim = config.embed_dim // config.n_heads
        self.scale = self.head_dim ** -0.5
        
        self.qkv = nn.Linear(config.embed_dim, 3 * config.embed_dim, bias=False)
        self.out_proj = nn.Linear(config.embed_dim, config.embed_dim, bias=False)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x, mask=None):
        B, S, _ = x.shape
        
        qkv = self.qkv(x).reshape(B, S, 3, self.n_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        
        if mask is not None:
            attn = attn.masked_fill(mask == 0, float('-inf'))
        
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        
        out = torch.matmul(attn, v).transpose(1, 2).reshape(B, S, -1)
        return self.out_proj(out)


class StandardBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.embed_dim)
        self.attn = StandardAttention(config)
        self.ln2 = nn.LayerNorm(config.embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(config.embed_dim, config.ff_dim),
            nn.GELU(),
            nn.Linear(config.ff_dim, config.embed_dim),
            nn.Dropout(config.dropout)
        )
    
    def forward(self, x, mask=None):
        x = x + self.attn(self.ln1(x), mask)
        x = x + self.ff(self.ln2(x))
        return x

In [13]:
config_standard = StandardConfig()
model_standard = StandardTransformer(config_standard).to(device)

# Use same training function but without resonance
def train_standard(model, config, train_dataset, val_dataset, n_epochs=2):
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=0.01)
    
    print(f"\nTraining: {config.name}")
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        
        for i, batch in enumerate(train_loader):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            
            loss = model(input_ids, labels=labels)["loss"]
            loss.backward()
            
            if (i + 1) % config.gradient_accumulation == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                optimizer.zero_grad()
            
            train_loss += loss.item()
            
            if (i + 1) % 500 == 0:
                print(f"    Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        avg_train = train_loss / len(train_loader)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                labels = batch["labels"].to(device)
                val_loss += model(input_ids, labels=labels)["loss"].item()
        
        avg_val = val_loss / len(val_loader)
        val_ppl = math.exp(avg_val)
        
        print(f"\n  Epoch {epoch+1}/{n_epochs}")
        print(f"    Train Loss: {avg_train:.4f}")
        print(f"    Val Loss: {avg_val:.4f}")
        print(f"    Val Perplexity: {val_ppl:.2f}")
    
    return {"val_ppl": [val_ppl]}

results_standard = train_standard(model_standard, config_standard, train_dataset, val_dataset, n_epochs=2)


Building model: standard_baseline
  Parameters: 123.81M

Training: standard_baseline
    Batch 500/25000, Loss: 8.1839
    Batch 1000/25000, Loss: 7.6808
    Batch 1500/25000, Loss: 7.5844
    Batch 2000/25000, Loss: 7.4999
    Batch 2500/25000, Loss: 7.3574
    Batch 3000/25000, Loss: 7.2238
    Batch 3500/25000, Loss: 7.4366
    Batch 4000/25000, Loss: 7.1390
    Batch 4500/25000, Loss: 7.3157
    Batch 5000/25000, Loss: 7.1293
    Batch 5500/25000, Loss: 7.3415
    Batch 6000/25000, Loss: 6.9636
    Batch 6500/25000, Loss: 6.9939
    Batch 7000/25000, Loss: 6.8041
    Batch 7500/25000, Loss: 6.9592
    Batch 8000/25000, Loss: 6.6607
    Batch 8500/25000, Loss: 6.5032
    Batch 9000/25000, Loss: 6.5045
    Batch 9500/25000, Loss: 6.2895
    Batch 10000/25000, Loss: 6.1241
    Batch 10500/25000, Loss: 6.4960
    Batch 11000/25000, Loss: 5.9056
    Batch 11500/25000, Loss: 6.4232
    Batch 12000/25000, Loss: 6.0449
    Batch 12500/25000, Loss: 6.1016
    Batch 13000/25000, Loss: 6.644